미세조정 전후 비교

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

test_reviews = [
    "This movie is absolutely amazing! Best film I've ever seen.",
    "Terrible waste of time. Worst movie ever made.",
    "It was okay, nothing special but not bad either.",
    "Brilliant acting and stunning visuals throughout!",
    "Boring and predictable plot. Very disappointed."
]

labels = ["긍정", "부정"]

# 미세 조정전 모델(랜덤 초기화)
BERT_MODEL_NAME = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL_NAME)
model_before = AutoModelForSequenceClassification.from_pretrained(BERT_MODEL_NAME)
model_before.eval()  #평가모드
with torch.no_grad():
    for i, review in enumerate(test_reviews):
        inputs = tokenizer(review,return_tensors='pt')
        outputs =  model_before(**inputs)
        logits = outputs.logits
        probs = torch.softmax(logits,dim=-1)[0]
        pred_label = torch.argmax(probs).item()

        print(f'리뷰 : {i+1} : {review}')
        print(f'예측 : {labels[pred_label]} {probs[pred_label]}')
        print(f'긍정 : {probs[0]} | 부정 : {probs[1]}\n')

In [ ]:
# 학습 - 미세조정
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
train_texts = [
    ("This is wonderful and fantastic!", 1),  # 긍정
    ("Absolutely terrible and awful!", 0),     # 부정
    ("I love this so much!", 1),
    ("Hated it completely!", 0)
]
BERT_MODEL_NAME = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL_NAME)
model_after = AutoModelForSequenceClassification.from_pretrained(BERT_MODEL_NAME,num_labels=2)
from transformers import AdamW
optimizer = AdamW(model_after.parameters(), lr=2e-5)
model_after.train()
for epoch in range(7):
    total_loss = 0
    for text,label in train_texts:
        optimizer.zero_grad()
        inputs = tokenizer(text,return_tensors='pt')
        labels = torch.tensor([label])
        outputs = model_after(**inputs,labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    avg_loss = total_loss / len(train_texts)
    print(f'epoch : {epoch+1}  loss : {avg_loss:.4f}')    

In [ ]:
print(f'미세조정 후')
test_reviews = [
    "This movie is absolutely amazing! Best film I've ever seen.",
    "Terrible waste of time. Worst movie ever made.",
    "It was okay, nothing special but not bad either.",
    "Brilliant acting and stunning visuals throughout!",
    "Boring and predictable plot. Very disappointed."
]

labels = ["긍정", "부정"]

with torch.no_grad():
    for i, review in enumerate(test_reviews):
        inputs = tokenizer(review,return_tensors='pt')
        outputs =  model_after(**inputs)
        logits = outputs.logits
        probs = torch.softmax(logits,dim=-1)[0]
        pred_label = torch.argmax(probs).item()        
        print(f'리뷰 : {i+1} : {review}')
        print(f'예측 : {labels[pred_label]} {probs[pred_label]}')
        print(f'긍정 : {probs[0]} | 부정 : {probs[1]}\n')